In [16]:
import numpy as np
from scipy.interpolate import CubicSpline
from tqdm import trange
import yt
yt.set_log_level(50)
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import utils
from utils import parse_input_file, grabFileSeries, get_simulation_time, dFdx_1d_non_U_grid, dFdx_mesh, dFdy_mesh

In [8]:
# Standard dataset initialization
index = 100
scratch_dirname = "vary_eta_force_free_IC/1e-4/"
# scratch_dirname = "forcetest/"
inputname = "athinput.recon_gauss_harris"
input_params = parse_input_file(f'/mnt/gs21/scratch/freem386/{scratch_dirname}{inputname}')
fileseries = grabFileSeries(scratch_dirname, index, basename="recon_fast")
ts = yt.DatasetSeries(fileseries)
time = get_simulation_time(fileseries[index])
ds = ts[index]

# Casting yt data into numpy data to analyze
# Defining analysis box limits (in code units of x and y)
xminbox, xmaxbox = 0, 1.2
yminbox, ymaxbox = 0, 0.1

# Creating the covering grid using YT to cast yt data
level = input_params["refinement1_level"]   # Max refinement level - highest resolution needed for numpy array
Nx = input_params["mesh_nx1"] * 2**(level)  # Nx, Ny is the number of cells in each direction of the WHOLE simulation domain
Ny = input_params["mesh_nx2"] * 2**(level)
xmin, xmax = input_params["mesh_x1min"], input_params["mesh_x1max"] # This is the dimension of the WHOLE simulation domain
ymin, ymax = input_params["mesh_x2min"], input_params["mesh_x2max"]
dx, dy = (xmax - xmin)/Nx, (ymax - ymin)/Ny     # grid resolution at highest refinement level
i0, j0 = int(np.floor((xminbox -  xmin  )//dx)), int(np.floor((yminbox -  ymin  )//dy))     # Index for starting x and y positions within the covering grid
iN, jN = int(np.floor((xmaxbox - xminbox)//dx)), int(np.floor((ymaxbox - yminbox)//dy))   # Length of the covering grid data in x/y

# Finally the real covering grid:
cg = ds.covering_grid(level=level, left_edge=[xminbox, yminbox, 0], dims=[iN, jN, 1])
# Grabbing mesh information using numpy
X, Y = np.reshape(cg["athena_pp","x"].v, (iN, jN)), np.reshape(cg["athena_pp","y"].v, (iN, jN))
x = X[:,0]
y = Y[0,:]

# Now grab relevant fluid quantities (sliced along x, y, then reducing dimension with [...,0], elminating YT units with .v)
vx = cg["gas", "velocity_x"][:,:,0].v
vy = cg["gas", "velocity_y"][:,:,0].v
Bx = cg["gas", "magnetic_field_x"][:,:,0].v
By = cg["gas", "magnetic_field_y"][:,:,0].v
vA = cg["gas", "alfven_speed"][:,:,0].v
rho =cg["gas", "density"][:,:,0].v

# Ez = u x B = eta*Jz, so current is:
J = (vx*By - vy*Bx) / input_params["problem_eta_ohm"]

In [21]:
fout, xout, _ = utils.collapse_refinement(By[:,0], x)
print(fout)
print("is strictly increasing?", np.all(np.diff(xout) > 0))
dBydx = dFdx_mesh(By, x)
dBxdy = dFdy_mesh(Bx, y)

/mnt/home/freem386/anaconda3/envs/research/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/home/freem386/anaconda3/envs/research/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[-2.3724052118648315e-06, -3.4013207500050085e-06, -4.430196838366736e-06, -5.4590362435721284e-06, -6.487841945995272e-06, -7.5166170229654146e-06, -8.545364539295731e-06, -9.574087796808184e-06, -1.0602790002263728e-05, -1.1631473429629976e-05, -1.2660140752728086e-05, -1.3688794106828387e-05, -1.4717437138436094e-05, -1.5746082285550452e-05, -1.6774732849211206e-05, -1.780338370703579e-05, -1.8832037496080085e-05, -1.986069895591075e-05, -2.088937174670075e-05, -2.1918059650815424e-05, -2.2946766388049096e-05, -2.3975495710120535e-05, -2.5004251352035116e-05, -2.603303682930384e-05, -2.7061855563189672e-05, -2.8090711222750456e-05, -2.9119607848953944e-05, -3.0148549579337317e-05, -3.1177540161925026e-05, -3.220658306872013e-05, -3.3235681384386396e-05, -3.426483813013921e-05, -3.529405689305632e-05, -3.632334145848307e-05, -3.735269572542874e-05, -3.838212379213173e-05, -3.941162993254633e-05, -4.044121840681272e-05, -4.147089317856966e-05, -4.250065768064534e-05, -4.35305148212527

dFdx:   0%|          | 0/959 [00:00<?, ?it/s]


ValueError: `x` must be strictly increasing sequence.